In [0]:
!pip install -q xlrd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hiv_report_data.xlsx to hiv_report_data (1).xlsx


In [0]:
#raw data
import pandas as pd
data = pd.read_excel('hiv_report_data.xlsx')

In [0]:
!pip install rake-nltk

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from rake_nltk import Rake
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [0]:

negative_cases_string = ""
negative = "wrongly  negative false "
r = Rake(language='english')
r.extract_keywords_from_text(negative)
tags = r.get_ranked_phrases()
for tag in tags:
    tokens = set(word_tokenize(tag))
    for token in tokens:
        curr_tag = stemmer.stem(token)
        if curr_tag not in death_string:
            negative_cases_string += curr_tag + " "

In [161]:
print(negative_cases_string)

fals neg wrong 


In [0]:
death_string = ""
deaths = "killed  death burned murder die "
r = Rake(language='english')
r.extract_keywords_from_text(deaths)
tags = r.get_ranked_phrases()
for tag in tags:
    tokens = set(word_tokenize(tag))
    for token in tokens:
        curr_tag = stemmer.stem(token)
        if curr_tag not in death_string:
            death_string += curr_tag + " "


In [137]:
print(death_string)

dea burn die murd kil 


In [0]:
suicide_string = ""
suicide = "jumped hanged  suicide "
r = Rake(language='english')
r.extract_keywords_from_text(suicide)
tags = r.get_ranked_phrases()
for tag in tags:
    tokens = set(word_tokenize(tag))
    for token in tokens:
        curr_tag = stemmer.stem(token)
        if curr_tag not in suicide_string:
            suicide_string += curr_tag + " "


In [139]:
print(suicide_string)

jump suicid hang 


In [0]:
articles=data.iloc[:,2].values
articles=list(articles)

In [0]:
#classifing negative reports articles
n=0
negative_article_no=[]
for article in articles:
    r = Rake(language='english')
    r.extract_keywords_from_text(article)
    tags = r.get_ranked_phrases()
    for tag in tags:
            tokens = set(word_tokenize(tag))
            for token in tokens:
                curr_tag = stemmer.stem(token)
                if curr_tag  in negative_cases_string:
                  negative_article_no.append(n)
                  
    n=n+1              
         

In [0]:
#classifing death articles
n=0
death_article_no=[]
for article in articles:
    r = Rake(language='english')
    r.extract_keywords_from_text(article)
    tags = r.get_ranked_phrases()
    for tag in tags:
            tokens = set(word_tokenize(tag))
            for token in tokens:
                curr_tag = stemmer.stem(token)
                if curr_tag  in death_string:
                  death_article_no.append(n)
                  
    n=n+1              
         

In [0]:
#classifing suicide articles
n=0
suicide_article_no=[]
for article in articles: 
    r = Rake(language='english')
    r.extract_keywords_from_text(article)
    tags = r.get_ranked_phrases()
    for tag in tags:
            tokens = set(word_tokenize(tag))
            for token in tokens:
                curr_tag = stemmer.stem(token)
                if curr_tag  in suicide_string:
                  suicide_article_no.append(n)
                  
    n=n+1              
         

In [0]:
death_dataset=pd.DataFrame(columns=['Date','Headline','Content'])
for n in death_article_no:
    death_dataset=death_dataset.append(pd.Series([data.iloc[n,1],data.iloc[n,2],data.iloc[n,3]], index=death_dataset.columns ), ignore_index=True)
        

In [0]:
suicide_dataset=pd.DataFrame(columns=['Date','Headline','Content'])
for n in suicide_article_no:
    suicide_dataset=suicide_dataset.append(pd.Series([data.iloc[n,1],data.iloc[n,2],data.iloc[n,3]], index=suicide_dataset.columns ), ignore_index=True)
        


In [0]:
negative_cases_dataset=pd.DataFrame(columns=['Date','Headline','Content'])
for n in negative_article_no:
    negative_case_dataset_dataset=negative_case_dataset_dataset.append(pd.Series([data.iloc[n,1],data.iloc[n,2],data.iloc[n,3]], index=negative_case_dataset_dataset.columns ), ignore_index=True)
        


In [0]:
a=0
new_data=pd.DataFrame(columns=['Class','Date','Headline','Content'])
while a<data.shape[0]:
  if (a in death_article_no) or (a in suicide_article_no) or (a in negative_article_no):
    pass
  else:
    new_data=new_data.append(pd.Series([data.iloc[a,0],data.iloc[a,1],data.iloc[a,2],data.iloc[a,3]], index=new_data.columns ), ignore_index=True)
  a=a+1 


In [0]:
classes=[1,2,3,4,5]
train_data=new_data.loc[new_data['Class'].isin(classes)]

In [0]:
processed_documents=[]
processed_class=[]
for _class in classes:
  class_articles=[]
  a=0  
  while a<train_data.shape[0]:
      if train_data.iloc[a,0]==_class:
          class_articles.append(train_data.iloc[a,2])
      a=a+1    
     
  for article in class_articles:
    processed_string="" 
    r = Rake(language='english')
    r.extract_keywords_from_text(article)
    tags = r.get_ranked_phrases()
    for tag in tags:
            tokens = set(word_tokenize(tag))
            for token in tokens:
                curr_tag = stemmer.stem(token)
                if curr_tag not in processed_string:
                    processed_string += curr_tag + " "
    processed_documents.append(processed_string)
    processed_class.append(_class)

In [0]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(processed_documents,processed_class,test_size=.2)


In [0]:
#Count Vector As A Feature
count_vect=CountVectorizer(analyzer='word')
count_vect.fit(processed_documents)

xtrain_count=count_vect.transform(xtrain)
xtest_count=count_vect.transform(xtest)

In [0]:
#word level tf-idf
tfidf_vect=TfidfVectorizer(analyzer='word',max_features=5000)
tfidf_vect.fit(processed_documents)

xtrain_tfidf=tfidf_vect.transform(xtrain)
xtest_tfidf=tfidf_vect.transform(xtest)

In [0]:
# ngram level tf-idf

tfidf_vect_ngram=TfidfVectorizer(analyzer='word',max_features=5000,ngram_range=(0,1))
tfidf_vect_ngram.fit(processed_documents)

xtrain_tfidf_ngram=tfidf_vect_ngram.transform(xtrain)
xtest_tfidf_ngram=tfidf_vect_ngram.transform(xtest)

In [0]:
def train_model(classifier,xtrain,ytrain,xtest,ytest):
    classifier.fit(xtrain,ytrain)
    predictions=classifier.predict(xtest)
    
    
    return metrics.accuracy_score(predictions,ytest)

In [0]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, ytrain, xtest_count,ytest)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, ytrain, xtest_tfidf,ytest)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on ngram level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, ytrain, xtest_tfidf_ngram,ytest)
print("NB, WordLevel TF-IDF: ", accuracy)

In [0]:
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [99]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
model = Sequential()
model.add(layers.Dense(64, input_dim=xtrain_tfidf.shape[1], activation='sigmoid'))
model.add(Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 64)                25920     
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 7)                 455       
Total params: 26,375
Trainable params: 26,375
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(xtrain_tfidf, ytrain,
                    batch_size=10,
                    epochs=1000,
                    validation_data=(xtest_tfidf, ytest),
                    verbose=1)